# Create Prompt Flows

---
## 1. Setup

Import dependencies

In [ ]:
import boto3

session = boto3.session.Session()
kendra = session.client("kendra")
iam_client = session.client("iam")
lambda_client = session.client("lambda")
bedrock_agents_clients = session.client("bedrock-agent")
agents_runtime_client = boto3.client("bedrock-agent-runtime")

---
## 2. Create Kendra retriever Lambda function

Create Lambda execution role

In [ ]:
import json


try:
    role = iam_client.create_role(
        RoleName="HASearchLambdaExecutionRole",
        AssumeRolePolicyDocument=json.dumps(
            {
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {"Service": "lambda.amazonaws.com"},
                        "Action": "sts:AssumeRole",
                    }
                ],
            }
        ),
    )
except:
    role = iam_client.get_role(RoleName="HASearchLambdaExecutionRole")

print(role)

iam_client.attach_role_policy(
    RoleName="HASearchLambdaExecutionRole",
    PolicyArn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
)

iam_client.put_role_policy(
    RoleName="HASearchLambdaExecutionRole",
    PolicyName="HASearchLambdaExecutionPolicy",
    PolicyDocument=json.dumps(
        {
            "Version": "2012-10-17",
            "Statement": [
                {"Effect": "Allow", "Action": ["kendra:query"], "Resource": ["*"]}
            ],
        }
    ),
)

Define Lambda function

In [ ]:
%%writefile retrieve_kendra.py

import boto3
import logging
import os

logger = logging.getLogger()
logger.setLevel("INFO")
boto_session = boto3.session.Session()
kendra = boto_session.client("kendra")


def convert_json_to_xml(json_data: list[dict]) -> str:
    """
    Convert JSON data to XML format for Anthropic Claude prompt.
    """

    docs = "<documents>"
    for i, snippet in enumerate(json_data, start=1):
        item = f"<document id='{i}'>"
        for k, v in snippet.items():
            item += f"<{k}>{v}</{k}>"
        item += "</document>"
        docs += item
    docs += "</documents>"
    return docs


def format_kendra_retrieve_response(result: dict) -> dict:
    """
    Format the Kendra retrieve response to a more readable format.
    """
    output = []
    for retrieve_result in result["ResultItems"]:
        if retrieve_result["ScoreAttributes"]["ScoreConfidence"] in [
            "LOW",
            "NOT_AVAILABLE",
        ]:
            continue
        item = {}
        item["Id"] = retrieve_result.get("DocumentId")
        item["Title"] = retrieve_result.get("DocumentTitle").get("Text")
        item["Content"] = retrieve_result.get("DocumentExcerpt").get("Text")
        item["Uri"] = retrieve_result.get("DocumentURI")
        item["Confidence"] = retrieve_result.get("ScoreAttributes").get(
            "ScoreConfidence"
        )

        for attribute in retrieve_result.get("DocumentAttributes"):
            if attribute.get("Key") == "_excerpt_page_number":
                item["Uri"] = (
                    item["Uri"]
                    + "#page="
                    + str(attribute.get("Value").get("LongValue"))
                )
                break

        for attribute in retrieve_result.get("DocumentAttributes"):
            if attribute.get("Key") in [
                "ApplicationNumber",
                "BrandName",
                "GenericName",
                "ManufacturerName",
                "Submission",
                "_category",
            ]:
                item[attribute["Key"].replace("_", "")] = attribute.get("Value").get(
                    "StringValue"
                )
        item["Category"] = item.pop("category")
        output.append(item)

    return output


def lambda_handler(event, context) -> dict:

    logger.info(event)

    query = event.get("node").get("inputs")[0].get("value")
    logging.info(query)

    kendra_index = event.get("kendra_index_id", os.environ["KENDRA_INDEX_ID"])
    page_size = event.get("page_size", 50)

    result = kendra.query(
        QueryText=query,
        IndexId=kendra_index,
        PageSize=page_size,
        QueryResultTypeFilter='DOCUMENT',
    )

    json_results = format_kendra_retrieve_response(result)
    logger.info(json_results)

    xml_results = convert_json_to_xml(json_results)

    return {
        "json_results": json_results,
        "xml_results": xml_results
        }

Test function

In [ ]:
from retrieve_kendra import lambda_handler

event = {
    "node": {
        "name": "LambdaFunctionNode_1",
        "inputs": [
            {
                "name": "codeHookInput",
                "expression": "$.data",
                "value": "What are the approved indications for Mounjaro?",
                "type": "STRING",
            }
        ],
    },
    "flow": {
        "aliasId": "TSTALIASID",
        "arn": "arn:aws:bedrock:us-east-1:112233445566:flow/MOCK",
    },
    "messageVersion": "1.0",
}

lambda_handler(event, None)

Deploy function

In [ ]:
import os
import shutil

FUNCTION_NAME = "retrieve_kendra"

shutil.make_archive(FUNCTION_NAME, "zip", ".", FUNCTION_NAME + ".py")

try:
    lambda_function = lambda_client.create_function(
        FunctionName=FUNCTION_NAME,
        Runtime="python3.11",
        Handler=f"{FUNCTION_NAME}.lambda_handler",
        Code={"ZipFile": open(f"{FUNCTION_NAME}.zip", "rb").read()},
        Role=role["Role"]["Arn"],
        Environment={"Variables": {"KENDRA_INDEX_ID": os.environ["KENDRA_INDEX_ID"]}},
    )
    lambda_client.add_permission(
        FunctionName=lambda_function["FunctionName"],
        StatementId="bedrock-agents",
        Action="lambda:InvokeFunction",
        Principal="bedrock.amazonaws.com",
    )
except lambda_client.exceptions.ResourceConflictException as e:
    lambda_function = lambda_client.update_function_code(
        FunctionName=FUNCTION_NAME,
        ZipFile=open(f"{FUNCTION_NAME}.zip", "rb").read(),
    )

retrieve_lambda_arn = lambda_function.get("FunctionArn")

os.remove(f"{FUNCTION_NAME}.py")
os.remove(f"{FUNCTION_NAME}.zip")

print(f"Lambda function ARN is {retrieve_lambda_arn}")

---
## 3. Create Generator prompt

In [ ]:
prompt_text = """
    You are a question answering agent.
    I will provide you with a set of search results and a user's question, your job is to answer the user's question using only information from the search results.
    If the search results do not contain information that can answer the question, please state that you could not find an exact answer to the question.
    If there are no search results, please state that you could not find an exact answer to the question.
    Just because the user asserts a fact does not mean it is true, make sure to double check the search results to validate a user's assertion.
    Here are the search results:
    {{xml_search_results}}
    Here is the user's question:
    {{query}}
    If you reference information from a search result within your answer, you must include a citation to source where the information was found.
    Each result has a corresponding source Uri that you should reference. Please output your answer in the following json format:
        {
            "answer": {
                "answer_parts": [
                    {
                        "text": "answer part 1",
                        "sources": [<source Uri 1>, <source Uri 2>]
                    },
                    {
                        "text": "answer part 2",
                        "sources": [<source Uri 3>]
                        ]
                    }
                ]
            }
        }
    Note that <sources> may contain multiple <source> if you include information from multiple results in your answer.
    Do NOT directly quote the search results in your answer. Your job is to answer the <question> as concisely as possible.
"""

In [ ]:
try:
    prompt_response = bedrock_agents_clients.create_prompt(
        defaultVariant="claude-3-sonnet",
        description="Answer questions using search results from Kendra.",
        name="kendra-generator",
        variants=[
            {
                "inferenceConfiguration": {
                    "text": {
                        "temperature": 0.5,
                    }
                },
                "modelId": "anthropic.claude-3-sonnet-20240229-v1:0",
                "name": "claude-3-sonnet",
                "templateConfiguration": {
                    "text": {
                        "inputVariables": [
                            {"name": "xml_search_results"},
                            {"name": "query"},
                        ],
                        "text": prompt_text,
                    }
                },
                "templateType": "TEXT",
            },
        ],
    )
except:
    existing_prompt_id = [
        prompt_summary.get("id")
        for prompt_summary in bedrock_agents_clients.list_prompts().get(
            "promptSummaries"
        )
        if prompt_summary.get("name") == "kendra-generator"
    ][0]
    prompt_response = bedrock_agents_clients.get_prompt(
        promptIdentifier=existing_prompt_id
    )

version_response = bedrock_agents_clients.create_prompt_version(
    promptIdentifier=prompt_response["id"]
)

generator_prompt_arn = version_response.get("arn")
print(f"Generator prompt ARN is {generator_prompt_arn}")

---
## 4. Create Format Answer Lambda Function

Define Lambda function

In [ ]:
%%writefile format_answer.py

import logging
import json

logger = logging.getLogger()
logger.setLevel("INFO")

def format_generate_response(response: str) -> str:
    """Format the response from the generate task"""
    import json
    import re

    output = []
    text = response
    text = re.sub(r"\n\s*", "", text)
    text = re.sub(r"(,|\n)]", "]", text)
    text = re.sub(r"(,|\n)}", "}", text)
    print(text)

    try:
        text = re.search(r"{.*}", text).group()
    except Exception as e:
        print(e)
        output.append({"text": "", "sources": []})
        return output
    for answer_part in json.loads(text)["answer"]["answer_parts"]:
        print(answer_part)
        part = {}
        part["text"] = answer_part["text"]
        part["sources"] = []
        for source in answer_part["sources"]:
            part["sources"].append(source)
            part["sources"] = list(set(part["sources"]))
        output.append(part)

    print(output)
    return output

def lambda_handler(event, context) -> dict:

    logger.info(event)

    answer_parts = format_generate_response(event.get("node").get("inputs")[0].get("value"))
    logger.info(answer_parts)
    return json.dumps(answer_parts)

Test function

In [ ]:
from format_answer import lambda_handler

example = """This is a test {"answer": {"answer_parts": [
{"text": "Mounjaro (tirzepatide) is indicated as an adjunct to diet and exercise to improve glycemic control in adults with type 2 diabetes mellitus.", 
"sources": ["http://www.accessdata.fda.gov/drugsatfda_docs/label/2022/215866s000lbl.pdf","http://www.accessdata.fda.gov/drugsatfda_docs/label/2023/215866Orig1s002s006lbl.pdf","https://www.accessdata.fda.gov/drugsatfda_docs/nda/2022/215866Orig1s000lbl.pdf"]},
{"text": "Mounjaro has limitations of use - it has not been studied in patients with a history of pancreatitis and is not indicated for use in patients with type 1 diabetes mellitus.", 
"sources": ["http://www.accessdata.fda.gov/drugsatfda_docs/label/2022/215866s000lbl.pdf", "http://www.accessdata.fda.gov/drugsatfda_docs/label/2023/215866Orig1s002s006lbl.pdf", "https://www.accessdata.fda.gov/drugsatfda_docs/nda/2022/215866Orig1s000lbl.pdf"]}
]}}"""

event = {
    "node": {
        "name": "LambdaFunctionNode_1",
        "inputs": [
            {
                "name": "codeHookInput",
                "expression": "$.data",
                "value": example,
                "type": "OBJECT",
            }
        ],
    },
    "flow": {
        "aliasId": "TSTALIASID",
        "arn": "arn:aws:bedrock:us-east-1:112233445566:flow/MOCK",
    },
    "messageVersion": "1.0",
}

lambda_handler(event, None)

Deploy function

In [ ]:
import os
import shutil

FUNCTION_NAME = "format_answer"

shutil.make_archive(FUNCTION_NAME, "zip", ".", FUNCTION_NAME + ".py")

try:
    lambda_function = lambda_client.create_function(
        FunctionName=FUNCTION_NAME,
        Runtime="python3.11",
        Handler=f"{FUNCTION_NAME}.lambda_handler",
        Code={"ZipFile": open(f"{FUNCTION_NAME}.zip", "rb").read()},
        Role=role["Role"]["Arn"],
        Environment={"Variables": {"KENDRA_INDEX_ID": os.environ["KENDRA_INDEX_ID"]}},
    )
    lambda_client.add_permission(
        FunctionName=lambda_function["FunctionName"],
        StatementId="bedrock-agents",
        Action="lambda:InvokeFunction",
        Principal="bedrock.amazonaws.com",
    )
except lambda_client.exceptions.ResourceConflictException as e:
    lambda_function = lambda_client.update_function_code(
        FunctionName=FUNCTION_NAME,
        ZipFile=open(f"{FUNCTION_NAME}.zip", "rb").read(),
    )

format_lambda_arn = lambda_function.get("FunctionArn")

os.remove(f"{FUNCTION_NAME}.py")
os.remove(f"{FUNCTION_NAME}.zip")

print(f"Lambda function ARN is {format_lambda_arn}")

---
## 5. Create Prompt Flow

Create IAM role

In [ ]:
import json


try:
    role = iam_client.create_role(
        RoleName="HASearchPromptFlowRole",
        AssumeRolePolicyDocument=json.dumps(
            {
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {"Service": "bedrock.amazonaws.com"},
                        "Action": "sts:AssumeRole",
                    }
                ],
            }
        ),
    )
except:
    role = iam_client.get_role(RoleName="HASearchPromptFlowRole")

print(role)

iam_client.put_role_policy(
    RoleName="HASearchPromptFlowRole",
    PolicyName="HASearchPromptFlowPolicy",
    PolicyDocument=json.dumps(
        {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": [
                        "bedrock:GetFlow",
                        "bedrock:GetPrompt",
                        "bedrock:InvokeModel",
                    ],
                    "Resource": ["*"],
                }
            ],
        }
    ),
)

In [ ]:
try:
    create_flow_response = bedrock_agents_clients.create_flow(
        definition={
            "connections": [
                {
                    "configuration": {
                        "data": {
                            "sourceOutput": "document",
                            "targetInput": "codeHookInput",
                        }
                    },
                    "name": "FlowInputNodeFlowInputNode0ToRetrieveLambdaFunctionNode0",
                    "source": "FlowInputNode",
                    "target": "Retrieve",
                    "type": "Data",
                },
                {
                    "configuration": {
                        "data": {
                            "sourceOutput": "functionResponse",
                            "targetInput": "xml_search_results",
                        }
                    },
                    "name": "RetrieveLambdaFunctionNode0ToGeneratePromptsNode0",
                    "source": "Retrieve",
                    "target": "Generate",
                    "type": "Data",
                },
                {
                    "configuration": {
                        "data": {
                            "sourceOutput": "functionResponse",
                            "targetInput": "document",
                        }
                    },
                    "name": "RetrieveLambdaFunctionNode0ToSearchOutputNode0",
                    "source": "Retrieve",
                    "target": "SearchOutputNode",
                    "type": "Data",
                },
                {
                    "configuration": {
                        "data": {"sourceOutput": "document", "targetInput": "query"}
                    },
                    "name": "FlowInputNodeFlowInputNode0ToGeneratePromptsNode1",
                    "source": "FlowInputNode",
                    "target": "Generate",
                    "type": "Data",
                },
                {
                    "configuration": {
                        "data": {
                            "sourceOutput": "modelCompletion",
                            "targetInput": "codeHookInput",
                        }
                    },
                    "name": "GeneratePromptsNode0ToFormatLambdaFunctionNode0",
                    "source": "Generate",
                    "target": "Format",
                    "type": "Data",
                },
                {
                    "configuration": {
                        "data": {
                            "sourceOutput": "functionResponse",
                            "targetInput": "document",
                        }
                    },
                    "name": "FormatLambdaFunctionNode0ToFlowOutputNodeFlowOutputNode0",
                    "source": "Format",
                    "target": "FlowOutputNode",
                    "type": "Data",
                },
            ],
            "nodes": [
                {
                    "configuration": {"input": {}},
                    "name": "FlowInputNode",
                    "outputs": [{"name": "document", "type": "String"}],
                    "type": "Input",
                },
                {
                    "configuration": {
                        "lambdaFunction": {"lambdaArn": retrieve_lambda_arn}
                    },
                    "inputs": [
                        {
                            "expression": "$.data",
                            "name": "codeHookInput",
                            "type": "String",
                        }
                    ],
                    "name": "Retrieve",
                    "outputs": [{"name": "functionResponse", "type": "Object"}],
                    "type": "LambdaFunction",
                },
                {
                    "configuration": {
                        "prompt": {
                            "sourceConfiguration": {
                                "resource": {"promptArn": generator_prompt_arn}
                            }
                        }
                    },
                    "inputs": [
                        {
                            "expression": "$.data.xml_results",
                            "name": "xml_search_results",
                            "type": "String",
                        },
                        {"expression": "$.data", "name": "query", "type": "String"},
                    ],
                    "name": "Generate",
                    "outputs": [{"name": "modelCompletion", "type": "String"}],
                    "type": "Prompt",
                },
                {
                    "configuration": {
                        "lambdaFunction": {"lambdaArn": format_lambda_arn}
                    },
                    "inputs": [
                        {
                            "expression": "$.data",
                            "name": "codeHookInput",
                            "type": "String",
                        }
                    ],
                    "name": "Format",
                    "outputs": [{"name": "functionResponse", "type": "String"}],
                    "type": "LambdaFunction",
                },
                {
                    "configuration": {"output": {}},
                    "inputs": [
                        {"expression": "$.data", "name": "document", "type": "String"}
                    ],
                    "name": "FlowOutputNode",
                    "type": "Output",
                },
                {
                    "configuration": {"output": {}},
                    "inputs": [
                        {
                            "expression": "$.data.json_results",
                            "name": "document",
                            "type": "Array",
                        }
                    ],
                    "name": "SearchOutputNode",
                    "type": "Output",
                },
            ],
        },
        description="Health authority search+RAG example using an Amazon Kendra knowledge base retriever.",
        executionRoleArn=role.get("Role").get("Arn"),
        name="HA-search-flow",
    )
except:
    existing_flow_id = [
        flow_summary.get("id")
        for flow_summary in bedrock_agents_clients.list_flows().get("flowSummaries")
        if flow_summary.get("name") == "HA-search-flow"
    ][0]
    create_flow_response = bedrock_agents_clients.get_flow(
        flowIdentifier=existing_flow_id
    )

In [ ]:
bedrock_agents_clients.get_flow(flowIdentifier=create_flow_response.get("id"))

In [ ]:
response = bedrock_agents_clients.prepare_flow(
    flowIdentifier=create_flow_response.get("id")
)
response

In [ ]:
flow_version_response = bedrock_agents_clients.create_flow_version(
    flowIdentifier=create_flow_response.get("id")
)

In [ ]:
try:
    flow_alias_response = bedrock_agents_clients.create_flow_alias(
        flowIdentifier=create_flow_response.get("id"),
        name="Health-Authority-QA",
        routingConfiguration=[
            {"flowVersion": flow_version_response.get("version")},
        ],
    )
except:
    existing_flow_alias = [
        flow_alias_summary.get("id")
        for flow_alias_summary in bedrock_agents_clients.list_flow_aliases(
            flowIdentifier=create_flow_response.get("id")
        ).get("flowAliasSummaries")
        if flow_alias_summary.get("name") == "Health-Authority-QA"
    ][0]
    print(existing_flow_alias)
    flow_alias_response = bedrock_agents_clients.get_flow_alias(
        aliasIdentifier=existing_flow_alias,
        flowIdentifier=create_flow_response.get("id"),
    )
flow_alias_response

---
## 6. Invoke prompt flow

In [ ]:
invoke_response = agents_runtime_client.invoke_flow(
    flowAliasIdentifier=flow_alias_response.get("id"),
    flowIdentifier=create_flow_response.get("id"),
    inputs=[
        {
            "content": {"document": "What are the approved indications for Mounjaro?"},
            "nodeName": "FlowInputNode",
            "nodeOutputName": "document",
        },
    ],
)

for event in invoke_response["responseStream"]:
    if "flowOutputEvent" in event:
        print(event["flowOutputEvent"]["content"]["document"])